# 🚀 InnovAIte – Colab Notebook

This notebook demonstrates the AI-powered content generation workflow using **LLaMA 2** for text generation and **Stable Diffusion** for image creation.

---

## 🔧 Setup & Installations

Install the required libraries.


In [ ]:
# Install essential libraries
!pip install diffusers transformers torch accelerate safetensors \
streamlit sentencepiece python-docx bitsandbytes --upgrade


In [ ]:
!pip install python-docx
!pip install bitsandbytes
!pip install --upgrade bitsandbytes
!pip install accelerate transformers


In [ ]:
from huggingface_hub import login

# Replace 'your_token_here' with your actual Hugging Face token
login("your_token_here")

In [ ]:
%%writefile app.py
import streamlit as st
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from diffusers import StableDiffusionPipeline
from PIL import Image
from io import BytesIO
from docx import Document
from docx.shared import Inches  # Added for proper image scaling

# Page Configurations (Must be the first Streamlit command)
st.set_page_config(page_title="AI-Powered Prompt Studio", layout="wide", initial_sidebar_state="expanded")

# model names
TEXT_MODEL = "meta-llama/Llama-2-13b-chat-hf"  # Upgraded model for better quality
IMAGE_MODEL = "stabilityai/stable-diffusion-2-1"  # Higher quality image model

# Load text generation model with 4-bit quantization
@st.cache_resource
def load_text_model():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True
    )
    tokenizer = AutoTokenizer.from_pretrained(TEXT_MODEL)
    model = AutoModelForCausalLM.from_pretrained(
        TEXT_MODEL,
        quantization_config=bnb_config,
        device_map="balanced"
    )
    return model, tokenizer

# Load image generation model
@st.cache_resource
def load_image_model():
    pipe = StableDiffusionPipeline.from_pretrained(
        IMAGE_MODEL,
        torch_dtype=torch.float16,
        device_map="balanced"
    )
    return pipe

# Load models
model, tokenizer = load_text_model()
pipe = load_image_model()

# Sidebar for Navigation
st.sidebar.title("Navigation")
pages = ["Home", "Prompt Optimization", "Text Generation", "Image Generation"]
selected_page = st.sidebar.radio("Go to", pages)

if selected_page == "Home":
    st.title("Welcome to InnovAIte")
    st.markdown("### AI-Powered Blog & Image Generation")
    st.markdown("""
**Why Use InnovAIte?**
- Generate high-quality blogs in seconds.
- Create stunning AI-generated images.
- Save time with automated content creation.
- Download ready-to-use content effortlessly.

**How It Works:**
1. **Enter a topic or idea**
2. **Generate optimized text**
3. **Generate relevant images**
4. **Download your content**

Explore the platform using the navigation sidebar to get started!
    """)

elif selected_page == "Prompt Optimization":
    st.title("Prompt Optimization")
    st.markdown("Refine your prompts for both text and image generation.")

    # Text Prompt Optimization
    st.markdown("### Blog Input Fields")
    user_topic = st.text_input("Enter your blog topic:", placeholder="E.g., Artificial Intelligence, Mental Health, Food Trends")
    tone = st.selectbox("Select the tone:", ["Formal", "Informal", "Creative", "Persuasive", "Inspirational"])
    audience = st.selectbox("Select the target audience:", ["Students", "Professionals", "General Audience", "Experts"])
    temperature = st.slider("Creativity Level (Temperature)", 0.1, 1.0, 0.7)
    response_length = st.selectbox("Response Length", ["Short (300–500 words)", "Medium (500–1000 words)", "Long (1000+ words)"])
    writing_style = st.selectbox("Writing Style", ["Storytelling", "Analytical", "Journalistic", "Technical", "Conversational"])

    if user_topic:
        text_prompt = f"Write a {tone} blog post about {user_topic} for {audience}. Creativity Level: {temperature}, Response Length: {response_length}, Writing Style: {writing_style}."
        st.write("### Generated Text Prompt:")
        st.code(text_prompt)

    # Image Prompt Optimization
    st.markdown("### Image Input Fields")
    image_subject = st.text_input("Describe the image subject:", placeholder="E.g., A futuristic cityscape, A serene mountain lake")
    image_style = st.selectbox("Select the image style:", ["Photorealistic", "Digital Art", "Watercolor", "Sketch", "Oil Painting"])
    lighting = st.selectbox("Lighting Conditions:", ["Natural", "Soft", "Dramatic", "Neon-lit", "Cinematic", "Sunset", "Nighttime"])
    camera_angle = st.selectbox("Camera Angle:", ["Close-up", "Wide-angle", "Overhead", "First-person view", "Isometric"])
    mood = st.selectbox("Mood & Emotion:", ["Mysterious", "Dreamy", "Futuristic", "Cozy", "Surreal", "Hyper-realistic"])
    color_scheme = st.selectbox("Color Scheme:", ["Black & White", "Pastel Tones", "Cyberpunk", "Vivid Colors", "Monochrome"])

    if image_subject:
        image_prompt = f"Generate a {image_style} image of {image_subject} with {lighting} lighting, {camera_angle} perspective, evoking a {mood} mood, and using a {color_scheme} color scheme."
        st.write("### Generated Image Prompt:")
        st.code(image_prompt)

elif selected_page == "Text Generation":
    st.title("AI-Powered Text Generation")
    text_prompt = st.text_input("Enter your prompt for text generation:")

    if st.button("Generate Text"):
        with st.spinner("Generating text..."):
            inputs = tokenizer(text_prompt, return_tensors="pt").to(model.device)
            output = model.generate(**inputs, max_new_tokens=500, temperature=0.7)
            result = tokenizer.decode(output[0], skip_special_tokens=True)
            st.markdown("### Generated Blog:")
            st.write(result)
elif selected_page == "Image Generation":
    st.title("AI-Powered Image Generation")
    image_prompt = st.text_input("Enter your prompt for image generation:")
    num_images = st.slider("Select number of images to generate", 1, 5, 1)

    if st.button("Generate Image(s)"):
        with st.spinner("Generating images..."):
            images = [pipe(image_prompt, guidance_scale=7.5, height=768, width=768).images[0] for _ in range(num_images)]
            st.session_state.generated_images = images

        for idx, image in enumerate(images):
            st.image(image, caption=f"Generated Image {idx + 1}", use_column_width=True)

# Download Button
if "generated_text" in st.session_state or "generated_images" in st.session_state:
    st.write("### Download Generated Content")
    if st.button("Download Word Document"):
        doc = Document()

        if "generated_text" in st.session_state:
            doc.add_heading("Generated Text", level=1)
            doc.add_paragraph(st.session_state.generated_text)

        if "generated_images" in st.session_state:
            doc.add_heading("Generated Images", level=1)
            for idx, image in enumerate(st.session_state.generated_images):
                image_stream = BytesIO()
                image.save(image_stream, format="PNG")
                image_stream.seek(0)
                doc.add_paragraph(f"Image {idx + 1}:")
                doc.add_picture(image_stream, width=Inches(4))  # Resizing images for better fit

        doc_io = BytesIO()
        doc.save(doc_io)
        doc_io.seek(0)

        st.download_button(
            label="Download Word File",
            data=doc_io,
            file_name="AI_Generated.docx",
            mime="application/vnd.openxmlformats-officedocument.wordprocessingml.document"
        )

if __name__ == "__main__":
    pass



In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501


---

## 📝 Text Generation

Use LLaMA 2 to generate detailed blog content from structured prompts.


---

## 🖼️ Image Generation

Use Stable Diffusion to create AI-generated visuals based on your prompts.


---

## 💾 Download Output

Download the generated blog content or images to your device.


---

## ✅ Summary

This notebook demonstrated:
- Blog generation using LLaMA 2
- Image creation using Stable Diffusion
- Export to downloadable formats

> Built with ❤️ using Python, Hugging Face, and Colab.
